<a href="https://colab.research.google.com/github/Yazeedx0/AI-Agent/blob/main/AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU crewai[tools,agentops]

In [ ]:
!pip install -qU tavily-python scrapegraph-py

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
import agentops
from pydantic import BaseModel, Field
from typing import List
from google.colab import userdata
from tavily import TavilyClient
from scrapegraph_py import Client
import os
import json


In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get("open-ai")
os.environ ["AGENTOPS_API_KEY"] = userdata.get("Agentops")
agentops.init(
    api_key=userdata.get('Agentops'),
    skip_auto_end_session=True
)

🖇 AgentOps: API Key is invalid: {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}.
	    Find your API key at https://app.agentops.ai/settings/projects
🖇 AgentOps: API Key is invalid: {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}.
	    Find your API key at https://app.agentops.ai/settings/projects
🖇 AgentOps: AgentOps has already been initialized. If you are trying to start a session, call agentops.start_session() instead.


In [ ]:
from re import search
import time
from genericpath import exists
output_dir = "./ai-agent-output"
os.makedirs(output_dir, exist_ok=True)

basic_llm=LLM(model="gpt-4o-mini", temperature=0)
search_client = TavilyClient(api_key=userdata.get('tavily'))
scrape_client=Client(api_key="sgai-de5b1ded-7a95-4933-8754-9eaf166e6895")

# **Agent A**

In [ ]:
import os
from typing import List
from pydantic import BaseModel, Field

no_keywords = 5
class SuggestedSearchQueries(BaseModel):
    queries: List[str] = Field(
        ...,
        title="Suggested search queries to be passed to the search engine",
        min_items=1,
        max_items=no_keywords
    )

search_queries_recommendation_agent = Agent(
    role="Search Queries Recommendation Agent",
    goal="\n".join([
        "The agent is designed to generate a diverse list of suggested search queries to be submitted to the search engine.",
        "These queries should focus on locating specific products efficiently and effectively."
    ]),
    backstory="The agent specializes in assisting product searches by providing tailored and optimized search query suggestions.",
    llm=basic_llm,
    verbose=True,
)

search_queries_recommendation_task = Task(
    description="\n".join([
        "Yazeed is tasked with identifying and purchasing a {product_name} at the most competitive price.",
        "The term 'best price' reflects the concept of 'value for strategy,' ensuring that the chosen product aligns with the company's strategic goals and delivers the highest return on investment.",
        "The target websites for purchasing include: {websites_list}.",
        "The objective is to identify all available products online for comprehensive comparison in a subsequent phase.",
        f"The selected stores must offer products within {{country_name}}.",
        f"Generate a maximum of {no_keywords} search queries.",
        "The Search keywords in {language}",
        "Each search query must lead to an e-commerce product page rather than blogs or generic listing pages."
    ]),
    expected_output="A JSON object containing a list of suggested search queries.",
    output_json=SuggestedSearchQueries,
    output_file=os.path.join(output_dir, "Step_1_Suggested_search_queries.json"),
    agent=search_queries_recommendation_agent
)

print("Task successfully defined!")


Task successfully defined!


# **Agent B**

In [ ]:
class SingleSearchResult(BaseModel):
  title:str
  url:str = Field(..., title="the page url")
  content:str
  score:float
  search_query:str


class ALLSearchResults(BaseModel):
  results: List[SingleSearchResult]
@tool
def search_engine_tool(query: str):
  """Usful for search-based queries. use this to find current information about any query related page using a search engine"""

  return search_client.search(query)

search_angine_agent = Agent(
    role= "Search Engine Agent",
    goal= "To Search for poducts based on the suggested queris",
    backstory = "The agent specializes in assisting product searches by providing tailored and optimized search query suggestions",
    llm = basic_llm,
    verbose=True,
    tools=[search_engine_tool]
)

search_engine_taks = Task(
    description="\n".join(["The task is to search for product based on the suggested search queries",
    "You have to collect results from multiple search queries",
    "Ignore any susbicious links or not an ecommerce website link.",
    "Ignore any search results with confidence score less than ({score_th}).",
    "The search results will be used to compare prices of products form different websites",


    ]),
    expected_output="A JSON Object containing the search resutls",
    output_jsont=ALLSearchResults,
    ouput_file=os.path.join(output_dir,"Step_2_search_results.json"),
    agent= search_angine_agent

)
print("Task successfully defined!")


Task successfully defined!


# **Agent C**

In [ ]:
class ProductSpecification(BaseModel):
    specification_name: str = Field(..., title="Name of the specification")
    specification_value: str = Field(..., title="Value of the specification")

class SingleExtractProduct(BaseModel):
    page_url: str = Field(..., title="The original URL of the product page")
    product_title: str = Field(..., title="Title of the product")
    product_url: str = Field(..., title="URL of the product")
    product_image_url: str = Field(..., title="URL of the product image")
    product_current_price: float = Field(..., title="Current price of the product")
    product_original_price: float = Field(..., title="Original price of the product")
    product_discount_percentage: str = Field(..., title="Discount percentage of the product")
    product_specs: List[ProductSpecification] = Field(..., title="List of product specifications")
    agent_recommendation_rank: int = Field(..., title="Recommendation rank given by the agent")
    agent_recommendation_notes: str = Field(..., title="Notes for the agent's recommendation")

class AllExtractProducts(BaseModel):
    products: List[SingleExtractProduct] = Field(..., title="List of all extracted products")
@tool
def web_scraping_tool(page_url: str, required_fields: list):
  """ An AI Tool to help an agent to scrape a web page
  Example:
   web_scraping_tool(
    page_url="",
    required_fields=["product_title", "current_price","old_price"]
   )
   """
  details = scrape_client.smartscraper(
      website_url=page_url,
      user_prompt="Extract " + json.dumps(required_fields, ensure_ascii=False) + " From the webpage",

  )
  return {
      "page_url":page_url,
      "detalis":details
  }


In [ ]:
scraping_agent = Agent(
    role="Web scraping agent",
    goal="To extract Details from any website",
    backstory="Agent is designed to help in looking for required values from any websites",
    llm=basic_llm,
    tools=[web_scraping_tool],
    verbose=False
)
scraping_task = Task(
    description="\n".join([
        "The task is to extract product details from any ecommecre stroe page url",
        "The task has to collect resuclt from multiple pages urls",

    ]),
    expected_output="A JSON Object containing products details",
    output_json=AllExtractProducts,
    output_file=os.path.join(output_dir,"Step_3_search_results.json"),
    agent=scraping_agent
)

# Run the AI Crew

In [ ]:
Yazeed_crew= Crew(
    agents=[
        search_queries_recommendation_agent,
        search_angine_agent,
        scraping_agent,

    ],
    tasks=[
        search_queries_recommendation_task,
        search_engine_taks,
        scraping_task,
    ],
    process=Process.sequential
)

In [ ]:
Yazeed_crew.kickoff(
    inputs={
        "product_name":"Caffee machine for the office",
        "websites_list":["www.amazon.com", "www.noon.com","www.jumia.com.eg"],
        "country_name": "Jordan",
        "no_keywords": "10",
        "language":"Engilsh",
        "score_th": 0.10,
        "top_recommendations_no": 10

    }
)